Prepare csv files downloaded from PubChem database

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv('/content/drive/MyDrive/Manuscrip_preparation/compounds/GLP1R_data2.csv')
df

,Unnamed: 0,PUBCHEM_CID,PUBCHEM_EXT_DATASOURCE_SMILES,PUBCHEM_ACTIVITY_OUTCOME
0,0,44290899.0,CC[C@H](C)[C@@H](C(=O)N[C@@H](C)C(=O)N[C@@H](C...,Active
1,1,122189729.0,CC[C@H](C)[C@@H](C(=O)N[C@@H](C)C(=O)N[C@@H](C...,Active
2,2,122189730.0,CCCCCCCCCCCCCCCCCC(=O)N[C@H](CCC(=O)NCCCC[C@@H...,Active
3,3,122189731.0,CCCCCCCCCCCCCCCCCCCC(=O)N[C@@H](CCC(=O)NCCOCCO...,Active
4,4,122189732.0,CC[C@H](C)[C@@H](C(=O)N[C@@H](C)C(=O)N[C@@H](C...,Active
...,...,...,...,...
862,862,604525.0,CC1=C(C(=NN1CC(=O)NN)C)Br,Inactive
863,863,2961794.0,CC(C(=O)NC(CC1=CC=C(C=C1)O)C(=O)OC)OC2=CC=CC=C2,Inactive
864,864,752521.0,CC1(N=C(NC(=N1)NC2=CC(=CC=C2)Br)N)C,Inactive
865,865,12006165.0,CCSC1=NN=C(C(=O)N1COC(=O)C2=CC=C(C=C2)S(=O)(=O...,Inactive


In [ ]:
!pip install rdkit-pypi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 29.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
import numpy as np
import scipy.stats as stats

# Load the data
df = pd.read_csv('/content/drive/MyDrive/Manuscrip_preparation/compounds/GLP1R_data2.csv')

# Initialize lists to store the calculated descriptors
num_aromatic_bonds = []
num_oxygen_bonds = []
num_acidic_groups = []
molecular_weights = []
logP_values = []
num_rotatable_bonds = []
num_hbond_donors = []
num_hbond_acceptors = []
TPSA_values = []
num_rings = []
is_active = df['PUBCHEM_ACTIVITY_OUTCOME'].tolist()

# Define functions for counting bonds and groups
def count_aromatic_bonds(mol):
    return sum([bond.GetIsAromatic() for bond in mol.GetBonds()])

def count_oxygen_atoms(mol):
    return sum([1 for atom in mol.GetAtoms() if atom.GetSymbol() == 'O'])

def count_acidic_groups(mol):
    acidic_groups = ['C(=O)[OH]', '[OH]C(=O)', 'O=C[OH]']  # Example acidic groups
    return sum([mol.HasSubstructMatch(Chem.MolFromSmarts(group)) for group in acidic_groups])

# Loop through the SMILES and calculate the descriptors
for smile in df['PUBCHEM_EXT_DATASOURCE_SMILES']:
    mol = Chem.MolFromSmiles(smile)
    num_aromatic_bonds.append(count_aromatic_bonds(mol))
    num_oxygen_bonds.append(count_oxygen_atoms(mol))
    num_acidic_groups.append(count_acidic_groups(mol))
    molecular_weights.append(Descriptors.MolWt(mol))
    logP_values.append(Descriptors.MolLogP(mol))
    num_rotatable_bonds.append(Descriptors.NumRotatableBonds(mol))
    num_hbond_donors.append(Descriptors.NumHDonors(mol))
    num_hbond_acceptors.append(Descriptors.NumHAcceptors(mol))
    TPSA_values.append(Descriptors.TPSA(mol))
    num_rings.append(Descriptors.RingCount(mol))

# Convert lists to Pandas DataFrame
descriptor_data = pd.DataFrame({
    'Aromatic Bonds': num_aromatic_bonds,
    'Oxygen Atoms': num_oxygen_bonds,
    'Acidic Groups': num_acidic_groups,
    'Molecular Weight': molecular_weights,
    'LogP': logP_values,
    'Rotatable Bonds': num_rotatable_bonds,
    'H-bond Donors': num_hbond_donors,
    'H-bond Acceptors': num_hbond_acceptors,
    'TPSA': TPSA_values,
    'Rings': num_rings,
    'Activity': is_active
})

# Separate active and inactive data
active_data = descriptor_data[descriptor_data['Activity'] == 'Active']
inactive_data = descriptor_data[descriptor_data['Activity'] == 'Inactive']

# Calculate means
active_means = active_data.mean(numeric_only=True)
inactive_means = inactive_data.mean(numeric_only=True)

# Perform Mann–Whitney U-test and get p-values
p_values = []
for descriptor in active_means.index:
    stat, p = stats.mannwhitneyu(active_data[descriptor], inactive_data[descriptor])
    p_values.append(p)

# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Descriptor': active_means.index,
    'Active Mean': active_means.values,
    'Inactive Mean': inactive_means.values,
    'p-value': p_values
})

# Print the summary table
print(summary_df, )

# Save the summary table to a CSV file
summary_df.to_csv('descriptor_summary.csv', index=False)


         Descriptor  Active Mean  Inactive Mean       p-value
0    Aromatic Bonds    27.299401      12.861429  6.095506e-40
1      Oxygen Atoms    38.706587       3.191429  3.947872e-42
2     Acidic Groups     2.065868       0.188571  1.450276e-78
3  Molecular Weight  2779.073012     371.189594  3.284883e-43
4              LogP    -7.254690       2.937599  1.854830e-35
5   Rotatable Bonds    92.353293       5.077143  2.735010e-33
6     H-bond Donors    37.461078       1.165714  8.193200e-37
7  H-bond Acceptors    38.365269       4.920000  2.183758e-57
8              TPSA  1106.795749      76.344429  2.055522e-44
9             Rings     5.592814       3.095714  8.917980e-39
